In [ ]:
import math

def estimate_slurm_resources(sim, n_inhom: int, n_times: int, n_batches: int) -> tuple[str, str]:
    """Estimate SLURM memory and time requirements based on workload."""
    combos = n_times * n_inhom
    num_combos_per_batch = combos // n_batches
    
    # Estimate memory: base 0.3G + factor for data size (complex64 = 8 bytes)
    len_t = len(sim.times_local) # actually it saves only a portion of this len: t_det up to time_cut 
    # -> so len_t is most important for time
    mem_gb = 0.3 + (num_combos_per_batch * len_t * 8 * 30) / (1024**3) # 30 is a safety factor
    requested_mem_gb = math.ceil(mem_gb * 10) / 10
    requested_mem_mb = int(requested_mem_gb * 1024)
    requested_mem = f"{requested_mem_mb}M"
    
    
    # Estimate time: scale based on solver, n_atoms, len_coh_times
    solver = sim.simulation_config.ode_solver
    n_atoms = sim.system.n_atoms
    base_time_per_combo_seconds = 1.5  # normalized from example: 1 combo in 3s for ME with 1 atom, 1 t_coh value for len_t = 1000
    if solver == "BR":
        base_time_per_combo_seconds = 2.5  # for len_t=1000, 1 combo, 1 atom, 1 t_coh
    base_time_per_combo_seconds *= n_atoms ** 2  # quadratic scaling with n_atoms (matrix diagonalization)
    base_time_per_combo_seconds *= len_t / 1000  # scaling with detection time length
    time_seconds = num_combos_per_batch * base_time_per_combo_seconds
    time_hours = max(0.1, time_seconds / 3600)  # minimum ~36 seconds for safety
    if time_hours < 1:
        minutes = int(time_hours * 60)
        requested_time = f"00:{minutes:02d}:00"
    else:
        days = int(time_hours) // 24
        hours = int(time_hours) % 24
        if days > 0:
            requested_time = f"{days}-{hours:02d}:00:00"
        else:
            requested_time = f"{hours:02d}:00:00"
    
    return requested_mem, requested_time

In [2]:
# Mock simulation object for testing
class MockSim:
    def __init__(self, ode_solver, n_atoms, len_t=1000):
        self.simulation_config = type('obj', (object,), {'ode_solver': ode_solver})
        self.system = type('obj', (object,), {'n_atoms': n_atoms})
        self.times_local = [0] * len_t  # mock times_local array

In [3]:
test_cases = [
    ('Currently i want to simulate: 2D (n_times=600) ME, 1 atom, length of times approx 1000', MockSim(ode_solver="ME", n_atoms=1, len_t=1000), 1, 600, 20),
]

In [4]:
# Run tests
print("SLURM Resource Estimation Test Results")
print("=" * 60)
for desc, sim, n_inhom, n_times, n_batches in test_cases:
    mem, time = estimate_slurm_resources(sim, n_inhom, n_times, n_batches)
    combos = n_times * n_inhom
    combos_per_batch = combos // n_batches
    print(f"{desc}:")
    print(f"  Total combos: {combos}, Combos/batch: {combos_per_batch}, N_inhom: {n_inhom}, N_times: {n_times}, N_batches: {n_batches}")
    print(f"  Solver: {sim.simulation_config.ode_solver}, N_atoms: {sim.system.n_atoms}")
    print(f"  Estimated Memory: {mem}, Time: {time}")
    print()

SLURM Resource Estimation Test Results
Currently i want to simulate: 2D (n_times=600) ME, 1 atom, length of times approx 1000:
  Total combos: 600, Combos/batch: 30, N_inhom: 1, N_times: 600, N_batches: 20
  Solver: ME, N_atoms: 1
  Estimated Memory: 409M, Time: 00:06:00



## Test Results Explanation

All tests use len_t=1000 (detection time points), num_combos_per_batch=50.

- **1D cases** (len_coh_times=1): Fast, minimal scaling.
- **2D cases**: Scale with len_coh_times (100 or 500).
- **BR vs ME**: BR has 20x time factor.
- **Atom count**: Quadratic scaling with n_atoms.

Adjust base values and factors based on real benchmarks!

# Testing SLURM Resource Estimation

This notebook tests the `estimate_slurm_resources` function with various parameter combinations to understand expected memory and time requirements.